# Setup

In [147]:
from json import loads
import pandas as pd
import os

def is_beer(article_name: str) -> bool:
    """
    Check if the article is a beer
    :param article_name: The name of the article
    :return: True if the article is a beer, False otherwise
    """
    banned = [
        "assiette", "plat", "régio", "cruche", "tu claques", "insigne", "verre", "mijole", "fais", "polo", "gueule",
        "t-shirt", "15n", "caca", "affond", "gobelet", "action", "souper"
        ]
    if any(article_name.lower().startswith(b) for b in banned):
        return False
    return True

def create_dir_if_absent(dir: str) -> None:
    if not os.path.exists(dir):
        os.makedirs(dir)

# Main function

In [148]:
# Load the JSON file
YEAR="2024-2025-Q2"
json = loads(open(f"db/{YEAR}.json").read())

dic = dict()
for key, article in json["articles"]["mapValue"]["fields"].items():

    fields = article["mapValue"]["fields"]
    name = fields.get("name", {}).get("stringValue")

    if not is_beer(name):
        continue

    degree = fields.get("degree", {}).get("integerValue") or fields.get("degree", {}).get("doubleValue")

    if degree is None:
        degree = fields.get("degree", {}).get("doubleValue")

    price_out = fields.get("price_out", {}).get("doubleValue")

    if price_out is None:
        price_out = fields.get("price_out", {}).get("integerValue")

    volume = fields.get("format", {}).get("integerValue")

    if volume is None:
        volume = fields.get("format", {}).get("doubleValue")

    beer_type = fields.get("type", {}).get("stringValue")

    ratio =  (float(degree) * float(volume)) / float(price_out)

    dic[name] = {"Degree": float(degree), "Price": float(price_out), "Volume": float(volume), "Type": beer_type, "Ratio": float(ratio)}

# Create a DataFrame
df = pd.DataFrame(dic).T

# Sort the DataFrame by the ratio
df = df.sort_values(by=["Ratio", "Price", "Volume"], ascending=[False, True, False])
create_dir_if_absent(f"csv/{YEAR}")
df.to_csv(f"csv/{YEAR}/ranker.csv")

# Top 20, any type of beer

In [149]:
df.head(20)

,Degree,Price,Volume,Type,Ratio
Kasteel triple,11.0,2.2,33.0,Blonde,165.0
Bush 10,10.5,2.2,33.0,Blonde,157.5
Piraat,10.5,2.2,33.0,Ambrée,157.5
Kerel Kaishaku Fût,15.0,2.5,25.0,Blonde,150.0
Corne du bois des pendus quadruple,12.0,2.7,33.0,Blonde,146.666667
Westmalle triple,9.5,2.2,33.0,Trappiste,142.5
Goliath triple,9.0,2.1,33.0,Blonde,141.428571
Chimay rouge,9.0,2.1,33.0,Trappiste,141.428571
Kerel Kaishaku,15.0,3.5,33.0,Blonde,141.428571
Gulden Draak,10.7,2.5,33.0,Brune,141.24


# Top 20, blonde beers

In [150]:
blonde_beers = df[df["Type"] == "Blonde"]
blonde_beers.to_csv(f"csv/{YEAR}/blonde_ranker.csv")
blonde_beers.head(20)

,Degree,Price,Volume,Type,Ratio
Kasteel triple,11.0,2.2,33.0,Blonde,165.0
Bush 10,10.5,2.2,33.0,Blonde,157.5
Kerel Kaishaku Fût,15.0,2.5,25.0,Blonde,150.0
Corne du bois des pendus quadruple,12.0,2.7,33.0,Blonde,146.666667
Goliath triple,9.0,2.1,33.0,Blonde,141.428571
Kerel Kaishaku,15.0,3.5,33.0,Blonde,141.428571
Filou,8.5,2.1,33.0,Blonde,133.571429
Abeille,8.0,2.0,33.0,Blonde,132.0
Triple plaisir (la),8.0,2.0,33.0,Blonde,132.0
Corne du bois des pendus 10 triple,10.0,2.5,33.0,Blonde,132.0


# Top 20, ambrées

In [151]:
amber_beers = df[df["Type"] == "Ambrée"]
amber_beers.to_csv(f"csv/{YEAR}/amber_ranker.csv")
amber_beers.head(20)

,Degree,Price,Volume,Type,Ratio
Piraat,10.5,2.2,33.0,Ambrée,157.5
Bush 12 Fût,12.0,2.2,25.0,Ambrée,136.363636
Coucou puissant,9.5,2.3,33.0,Ambrée,136.304348
Gulden Draak 9000,10.5,2.6,33.0,Ambrée,133.269231
Bush 12,12.0,7.0,75.0,Ambrée,128.571429
Carolus ambrio,8.0,2.2,33.0,Ambrée,120.0
Troubadour magma,9.0,2.5,33.0,Ambrée,118.8
Satan red,8.0,2.3,33.0,Ambrée,114.782609
Bon secours heritage,8.0,2.3,33.0,Ambrée,114.782609
Maredsous 10,10.0,6.6,75.0,Ambrée,113.636364


# Top trappistes

In [152]:
trapist_beers = df[df["Type"] == "Trappiste"]
trapist_beers.to_csv(f"csv/{YEAR}/trapist_ranker.csv")
trapist_beers

,Degree,Price,Volume,Type,Ratio
Westmalle triple,9.5,2.2,33.0,Trappiste,142.5
Chimay rouge,9.0,2.1,33.0,Trappiste,141.428571
Rochefort 10,11.3,2.7,33.0,Trappiste,138.111111
Chimay blanche,8.0,2.0,33.0,Trappiste,132.0
Rochefort 8,9.2,2.3,33.0,Trappiste,132.0
Rochefort triple extra,9.0,2.3,33.0,Trappiste,129.130435
Rochefort 6,7.5,2.1,33.0,Trappiste,117.857143
Westmalle double,7.0,2.0,33.0,Trappiste,115.5
Chimay grande reserve (bleue),9.0,6.0,75.0,Trappiste,112.5
Chimay Cinq Cents(gde blanc),8.0,5.5,75.0,Trappiste,109.090909


# Top 20, brunes

In [153]:
brown_beers = df[df["Type"] == "Brune"]
brown_beers.to_csv(f"csv/{YEAR}/brown_ranker.csv")
brown_beers.head(20)

,Degree,Price,Volume,Type,Ratio
Gulden Draak,10.7,2.5,33.0,Brune,141.24
Saint feuillien brune,8.5,2.0,33.0,Brune,140.25
Saint Bernardus Abr 12,10.0,2.4,33.0,Brune,137.5
Kasteel donker,11.0,2.7,33.0,Brune,134.444444
Carolus classic,8.5,2.1,33.0,Brune,133.571429
Silly Scotch,8.0,2.0,33.0,Brune,132.0
Straffe Hendrik quadrupel,11.0,2.8,33.0,Brune,129.642857
Malheur 12,12.0,3.1,33.0,Brune,127.741935
Moinette brune,8.5,5.0,75.0,Brune,127.5
Lupulus brune,8.5,5.0,75.0,Brune,127.5


# Top 20, fruitées

In [154]:
fruit_beers = df[df["Type"] == "Fruitée"]
fruit_beers.to_csv(f"csv/{YEAR}/fruit_ranker.csv")
fruit_beers.head(20)

,Degree,Price,Volume,Type,Ratio
Queue de charrue rouge,8.7,2.3,33.0,Fruitée,124.826087
Gauloise fruits rouges,8.2,2.2,33.0,Fruitée,123.0
Frambush,8.5,2.4,33.0,Fruitée,116.875
Barbar Rouge,8.0,2.4,33.0,Fruitée,110.0
Val dieu fruitee,9.0,2.7,33.0,Fruitée,110.0
Tête de mort red,8.2,2.7,33.0,Fruitée,100.222222
Pêche Mel Bush Fût,8.0,2.0,25.0,Fruitée,100.0
Chouffe cherry Fût,8.0,2.0,25.0,Fruitée,100.0
Delirium red Fût,8.0,2.0,25.0,Fruitée,100.0
Bon secours myrtille,7.0,2.4,33.0,Fruitée,96.25


# Top blanches

In [155]:
white_beers = df[df["Type"] == "Blanche"]
white_beers.to_csv(f"csv/{YEAR}/white_ranker.csv")
white_beers

,Degree,Price,Volume,Type,Ratio
Bon secours prestige,9.0,2.5,33.0,Blanche,118.8
St Hubertus Blanche Fût,7.2,2.0,25.0,Blanche,90.0
Chouffe blanche,6.0,2.2,33.0,Blanche,90.0
Blanche de Bruxelles,4.5,1.8,33.0,Blanche,82.5
Peak blanche IPA,6.5,2.6,33.0,Blanche,82.5
St bernardus witbier,5.5,2.5,33.0,Blanche,72.6
Bornier vice,6.0,2.9,33.0,Blanche,68.275862
Troublette,5.6,2.8,33.0,Blanche,66.0
Blanche de Namur,4.5,1.8,25.0,Blanche,62.5
Arsenal,5.4,2.9,33.0,Blanche,61.448276


# Top 75cl

In [156]:
big = df[df["Volume"]  == 75]
big.to_csv(f"csv/{YEAR}/75cl_ranker.csv")
big

,Degree,Price,Volume,Type,Ratio
Bush 12,12.0,7.0,75.0,Ambrée,128.571429
Moinette brune,8.5,5.0,75.0,Brune,127.5
Lupulus brune,8.5,5.0,75.0,Brune,127.5
Lupulus Hibernatus,9.0,5.5,75.0,Brune,122.727273
Triple Moine,7.3,4.7,75.0,Blonde,116.489362
Moinette blonde,8.5,5.5,75.0,Blonde,115.909091
Lupulus blonde,8.5,5.5,75.0,Blonde,115.909091
Lupulus Organicus,8.5,5.5,75.0,Blonde,115.909091
Maredsous 10,10.0,6.6,75.0,Ambrée,113.636364
Chimay grande reserve (bleue),9.0,6.0,75.0,Trappiste,112.5
